In [1]:
import gym
import numpy as np

Documentation on Github: https://github.com/openai/gym/blob/master/gym/envs/classic_control/mountain_car.py

In [2]:
env = gym.make("CartPole-v0")

In [3]:
#env.reset()
#for _ in range(1000):
#    env.render()
#    env.step(env.action_space.sample()) # take a random action
#env.close()

In [4]:
class agent:
    
    def __init__(self):
        self.env = gym.make("CartPole-v0")
        self.env.reset()
        
    def select_action(self,theta):
        action_list = self.env.action_space
        weights = [policy(self.env.state, action, action_list, theta) for action in get_space_list(action_list)]
        return np.random.choice([action for action in get_space_list(action_list)],p = weights)
        
    def Reinforce(self, alpha, gamma, nr_episodes):
        theta = np.zeros(len(feature_vector(self.env.state,self.env.action_space.sample())))
        
        for k in range(nr_episodes):
            self.env.reset()
            S = [self.env.state]
            a = self.select_action(theta)
            A = [a]
            R = []
            new_state, reward, boolean,klam = self.env.step(a)
            S.append(new_state)
            R.append(reward)
            
            while not boolean:
                a = self.select_action(theta)
                A.append(a)
                new_state, reward, boolean, klam = self.env.step(a)
                S.append(new_state)
                R.append(reward)
            
            for t in range(len(A)):
                G = 0
                for j in range(t,len(A)):
                    G += gamma**(j-t-1)*R[j]
                gradient = (
                    feature_vector(S[t],A[t])-
                    sum([policy(S[t],b,self.env.action_space,theta)*feature_vector(S[t],b) for b in get_space_list(self.env.action_space)],0)
                )
                theta += alpha*(gamma**t)*G*gradient
        return theta
    
    
def policy(state,action,action_space,theta):
    denom = sum([np.exp(theta@feature_vector(state,a)) for a in get_space_list(action_space)],0)
    return np.exp(theta@feature_vector(state,action))/denom   
    
def feature_vector(state, action):
    s = state
    x = np.array([1,s[0],s[1],s[2],s[3],s[0]*s[1],s[0]*s[2],s[0]*s[3],s[1]*s[2],s[1]*s[3],s[2]*s[3], action])
    return x

In [5]:
def get_space_list(space):

    """
    Converts gym space, constructed from types, to list space_list
    """

    # -------------------------------- #

    types = [
        gym.spaces.multi_binary.MultiBinary,
        gym.spaces.discrete.Discrete,
        gym.spaces.multi_discrete.MultiDiscrete,
        gym.spaces.dict.Dict,
        gym.spaces.tuple.Tuple,
    ]

    if type(space) not in types:
        raise ValueError(f'input space {space} is not construdted from spaces of types:' + '\n' + str(types))

    # -------------------------------- #

    if type(space) is gym.spaces.multi_binary.MultiBinary:
        return [
            np.reshape(np.array(element), space.n)
            for element in itertools.product(
                *[range(2)] * np.prod(space.n)
            )
        ]

    if type(space) is gym.spaces.discrete.Discrete:
        return list(range(space.n))

    if type(space) is gym.spaces.multi_discrete.MultiDiscrete:
        return [
            np.array(element) for element in itertools.product(
                *[range(n) for n in space.nvec]
            )
        ]

    if type(space) is gym.spaces.dict.Dict:

        keys = space.spaces.keys()
        
        values_list = itertools.product(
            *[get_space_list(sub_space) for sub_space in space.spaces.values()]
        )

        return [
            {key: value for key, value in zip(keys, values)}
            for values in values_list
        ]

        return space_list

    if type(space) is gym.spaces.tuple.Tuple:
        return [
            list(element) for element in itertools.product(
                *[get_space_list(sub_space) for sub_space in space.spaces]
            )
        ]

    # -------------------------------- #

In [6]:
ag = agent()

In [7]:
ag.Reinforce(0.1,0.9,100)

array([ 4.76903342e-15,  1.48381613e-17,  7.35774310e-16, -8.74443689e-18,
        1.61539419e-16,  1.10724807e-17,  1.89970456e-18, -3.07927255e-17,
       -4.58467036e-17, -9.64873285e-17,  1.83135937e-17, -4.58528130e+00])